# Group 7
## Questions
1. How many of the world's 1-year old children today have been vaccinated against some disease?
2. How many against more diseases?
3. How has the rate of vaccination for different diseases changed over time?
4. Are there country characteristics that predict vaccination levels, or trends in vaccination levels?

## Datasets
1. The data for the immunization coverage among 1-year-olds is provided by thw Wolrd Health Organization (https://www.who.int/data/gho/gho-search?indexCatalogue=ghosearchindex&searchQuery=immunization%20coverage%20among%201-year-olds&wordsMode=AllWords)
2. ... to be continued for country characteristics

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import typing
import json

In [2]:
# Mount google drive if notebook is running in colab
if os.getenv("COLAB_RELEASE_TAG"):
  from google.colab import drive
  drive.mount("/content/drive", force_remount=True)
  IN_COLAB = True
else:
  IN_COLAB = False

In [3]:
# Depending on whether the notebook is running in google colab or not the base path for loading data is set accordingly
if IN_COLAB:
  base_data_path = os.path.join(os.curdir, 'drive', 'MyDrive', 'DOPP', 'data')
else:
  base_data_path = os.path.join(os.curdir, 'data')

## Helpers Functions

In [4]:
mapping = [
    ('WHS4_100', "diphtheria_pertussis_tetanus"),
    ('WHS4_129', "haemophilus_influenzae"),
    ('WHS4_117', "hepatitisB"),
    ('WHS8_110', "measles"),
    ('WHS4_544', "polio"),
    ('ROTAC', "rotavirus"),
    ('PCV3', "streptococcus_pneumoniae"),
    ('WHS4_543', "tuberculosis"),
]


def diseaseNameToVaccineIndicator(disease: str) -> str:
    for e in mapping:
        if e[1] == disease:
            return e[0]

    raise ValueError("Invalid disease")


def vaccineIndicatorToDiseaseName(indicator: str) -> str:
    for e in mapping:
        if e[0] == indicator:
            return e[1]

    raise ValueError("Invalid indicator")


def printDataDimensions(df: pd.DataFrame) -> None:
    print(f"Data dimensions are: {df.shape[0]} rows and {df.shape[1]} columns")
    print(f"\nindex types are: \n-----------------\n{df.index.dtypes}")
    print(f"\ncolumn types are: \n-----------------\n{df.dtypes}")

    print(
        f"\nFor daily data: \nData dimensions are: {df.shape[0]} rows and {df.shape[1]} columns")
    print(f"\nindex types are: \n-----------------\n{df.index.dtypes}")
    print(f"\ncolumn types are: \n-----------------\n{df.dtypes}")


# Load JSON country code mapping
country_mapping_path = os.path.join(base_data_path, 'country_code_mapping.json')
country_code_list = []
with open(country_mapping_path) as f:
    data = json.load(f)

    for item in data:
        country_data = {'alpha2': None, 'alpha3': None, 'country': None}
        country_data['alpha2'] = item['alpha2']
        country_data['alpha3'] = item['alpha3']
        country_data['country'] = item['country']
        country_code_list.append(country_data)

def getCountryMapping(search: str, returnKey: str, log=False):
    searchLc = search.lower()
    for item in country_code_list:
        if item['country'] == searchLc or item['alpha2'] == searchLc or item['alpha3'] == searchLc:
            return item.get(returnKey)

    if log :
      print(f'Invalid country {search}')
    return None


## 1. Load Data
### 1.1 Load Immunization Data

In [5]:
def load_immunization_data(immunization_data_path: str) -> pd.DataFrame:
  immunization_data_directory = glob.glob(os.path.join(immunization_data_path, '*.csv'))

  immunization_data_list = []
  for filename in immunization_data_directory:
      df = pd.read_csv(filename, sep=',') 
      indicator = df['IndicatorCode'][0]
      df.rename(columns={'Period': 'year', 'Location': 'country', 'Value': indicator }, inplace=True)
      df = df[['year', 'country', indicator]].copy()
      df['country'] = df['country'].str.lower()
      df.set_index(['year', 'country'], inplace=True)
      immunization_data_list.append(df)

  immunization_data = pd.concat(immunization_data_list, axis=1)
  
  immunization_data[diseaseNameToVaccineIndicator('diphtheria_pertussis_tetanus')] = immunization_data[diseaseNameToVaccineIndicator('diphtheria_pertussis_tetanus')].astype(float)
  immunization_data[diseaseNameToVaccineIndicator('measles')] = immunization_data[diseaseNameToVaccineIndicator('measles')].astype(float)

  immunization_data.sort_values(['year'], inplace=True)

  return immunization_data

In [6]:
immunization_data_path = os.path.join(base_data_path, 'immunization')
immunization_data = load_immunization_data(immunization_data_path)

display(immunization_data[immunization_data.index.get_level_values('country') == 'austria'])
# printDataDimensions(immunization_data)

,,WHS4_100,WHS4_129,WHS4_117,WHS8_110,WHS4_544,ROTAC,PCV3,WHS4_543
year,country,,,,,,,,
2000,austria,81.0,72.0,33.0,75.0,71.0,NaN,NaN,NaN
2001,austria,84.0,74.0,44.0,79.0,83.0,NaN,NaN,NaN
2002,austria,83.0,82.0,81.0,78.0,82.0,NaN,NaN,NaN
2003,austria,84.0,84.0,83.0,79.0,84.0,NaN,NaN,NaN
2004,austria,83.0,83.0,83.0,74.0,83.0,NaN,NaN,NaN
2005,austria,86.0,86.0,86.0,75.0,86.0,NaN,NaN,NaN
2006,austria,83.0,83.0,83.0,80.0,83.0,NaN,NaN,NaN
2007,austria,85.0,85.0,85.0,79.0,85.0,NaN,NaN,NaN
2008,austria,83.0,83.0,83.0,83.0,83.0,47.0,NaN,NaN


### 1.2 Load Infant Mortality Data

In [7]:
def load_infant_mortality_data(infant_mortality_data_path: str) -> pd.DataFrame:
    df = pd.read_csv(infant_mortality_data_path, sep=',') 
    df.rename(columns={'Period': 'year', 'Location': 'country', 'FactValueNumeric': 'infant_mortality'}, inplace=True)
    df = df.loc[df['Dim1'] == 'Both sexes']
    df = df[['year', 'country', 'infant_mortality']].copy()
    df['country'] = df['country'].str.lower()
    df.set_index(['year', 'country'], inplace=True)
    df.sort_values(['year', 'country'], inplace=True)

    return df


In [8]:
infant_mortality_data_path = os.path.join(base_data_path, 'country_characteristics', 'infant_mortality_rate.csv')
infant_mortality_data = load_infant_mortality_data(infant_mortality_data_path)

display(infant_mortality_data[infant_mortality_data.index.get_level_values('country') == 'austria'])
# printDataDimensions(infant_mortality_data)

,,infant_mortality
year,country,
1949,austria,83.37
1950,austria,73.12
1951,austria,64.59
1952,austria,57.84
1953,austria,52.72
...,...,...
2016,austria,2.97
2017,austria,2.93
2018,austria,2.92


### 1.3 Load Education Data

In [9]:
def load_education_data(education_data_path: str) -> pd.DataFrame:
    df = pd.read_csv(education_data_path, sep=',') 
    df.rename(columns={'TIME_PERIOD': 'year', 'geo': 'alpha2','OBS_VALUE': 'education_level'}, inplace=True) 
    df = df.loc[(df['sex'] == 'T') & (df['age'] == 'Y15-74') & (df['isced11'] == 'ED3-8')]
    df['alpha2'] = df['alpha2'].str.lower() 
    df = df.assign(country=lambda x: x.alpha2.map(lambda y: getCountryMapping(y, 'country')))
    df = df[df['country'].notna()]
    df = df[['year', 'country', 'education_level']].copy()
    df['country'] = df['country'].str.lower()
    df.set_index(['year', 'country'], inplace=True)
    df.sort_values(['year'], inplace=True)

    return df

In [10]:
education_data_path = os.path.join(base_data_path, 'country_characteristics', 'education.csv')
education_data = load_education_data(education_data_path)

display(education_data[education_data.index.get_level_values('country') == 'austria'])
# printDataDimensions(education_data)

,,education_level
year,country,
2004,austria,72.5
2005,austria,73.2
2006,austria,72.7
2007,austria,72.4
2008,austria,73.5
2009,austria,74.5
2010,austria,75.2
2011,austria,75.3
2012,austria,76.0


### 1.4 Load GDP Data

In [11]:
def load_gdp_data(gpd_data_path: str) -> pd.DataFrame:
    df = pd.read_csv(gpd_data_path, sep=',')
    df.rename(columns={'Country Code': 'alpha3'}, inplace=True)
    df['alpha3'] = df['alpha3'].str.lower()
    df = df[df['alpha3'].notna()]
    df = df.assign(country=lambda x: x.alpha3.map(lambda y: getCountryMapping(y, 'country')))
    df = df[df['country'].notna()]
    value_vars = [f'{str(x)} [YR{str(x)}]' for x in range(2000, 2022)]
    df = pd.melt(df, id_vars='country', var_name='year', value_name='gdp', value_vars=value_vars)
    df = df.assign(year=lambda x: x.year.str.split(' ').str[0])
    df['year'] = df['year'].astype(int)
    df.set_index(['year', 'country'], inplace=True)
    df.sort_values(['year', 'country'], inplace=True)

    return df


In [12]:
gpd_data_path = os.path.join(base_data_path, 'country_characteristics', 'gdp', 'gdp.csv')
gdp_data = load_gdp_data(gpd_data_path)

display(gdp_data[gdp_data.index.get_level_values('country') == 'austria'])
# printDataDimensions(gdp_data)

,,gdp
year,country,
2000,austria,29376.019736
2001,austria,29702.797529
2002,austria,31178.693330
2003,austria,32144.241805
2004,austria,33773.825777
2005,austria,35013.714044
2006,austria,37662.185166
2007,austria,39430.609168
2008,austria,41316.225177


### 1.5 Load Incidence Data

In [13]:
## TODO Data is spotty, need to figure out which values (val, upper, lower) we want

vaccineMapping = {
    'Acute hepatitis B': 'WHS4_117',
    'Total burden related to hepatitis B': 'WHS4_117',
    'Tuberculosis': 'WHS4_543',
    'Diarrheal diseases': 'ROTAC',
    'Measles': 'WHS8_110',
    'Whooping cough': None,
    'Tetanus': 'WHS4_100',
    'Diphtheria': 'WHS4_100',
}

def load_incidence_data(incidence_data_path: str) -> pd.DataFrame:
    df = pd.read_csv(incidence_data_path, sep=',')
    df.rename(columns={'cause_name': 'disease', 'val': 'incidence'}, inplace=True)
    df = df.assign(country=lambda x: x.location_name.map(lambda y: getCountryMapping(y, 'country')))
    df = df.assign(vaccine=lambda x: x.disease.map(lambda y: vaccineMapping.get(y)) )
    df = df.loc[(df['sex_name'] == 'Both') & (df['age_name'] == 'All ages')  & (df['metric_name'] == 'Number')]
    df = df[df['country'].notna()]
    df = df[['year', 'country', 'incidence', 'disease', 'vaccine']].copy()
    df.set_index(['year', 'country'], inplace=True)
    df.sort_values(['year', 'country'], inplace=True)
    
    return df


In [14]:
incidence_data_path = os.path.join(base_data_path, 'country_characteristics', 'incidence', 'incidence.csv')
incidence_data = load_incidence_data(incidence_data_path)

display(incidence_data[incidence_data.index.get_level_values('country') == 'austria'])
# printDataDimensions(incidence_data)

incidence                              disease   vaccine
year country                                                              
2000 austria   75509.658089  Total burden related to hepatitis B  WHS4_117
     austria  839469.083397                         Tuberculosis  WHS4_543
     austria   41324.236328                   Diarrheal diseases     ROTAC
     austria    1996.429470                    Acute hepatitis B  WHS4_117
     austria       0.003504                           Diphtheria  WHS4_100
...                     ...                                  ...       ...
2019 austria       7.243609                              Measles  WHS8_110
     austria  679883.146009                         Tuberculosis  WHS4_543
     austria   53078.380568                   Diarrheal diseases     ROTAC
     austria    1534.204973                    Acute hepatitis B  WHS4_117
     austria   69169.135757  Total burden related to hepatitis B  WHS4_117

[160 rows x 3 columns]

#To do
1. How many of the world's 1-year old children today have been vaccinated against some disease?
2. How many against more diseases?
3. How has the rate of vaccination for different diseases changed over time?

Zunächst reicht nur Österreich, wenn Zeit ist, können wir noch andere Länder hinzunehmen. Mit Seaborn kann man besonders schöne Plots erstellen, wer mag, muss aber nicht sein, matplotlib reicht auch
1. Plotten der Prozentzahl jeder Krankheit aktuell (also letzte Jahreszahl die wir haben, in dem Falle 2021) von Österreich
2. Wird mit 1 eigentlich beantwortet, wüsste nicht, was da noch mehr hingehört
3. Plotten der Prozentzahl jeder Krankheit über die Zeit von Österreich

Wenn fertig, kann man ja noch andere Länder nehmen. Frankreich oder Italieren würde sich womöglich anbieten

## Are there country characteristics that predict vaccination levels, or trends in vaccination levels?
Testing correlations between certain country characteristics and vaccination levels. Can we achieve accurate predictions? We will use logistic regression. Used country characteristics (Austria): education level, infant mortality rate, (gdp per capita), vaccine mandates

The number of diseases we are looking at is 8

## TO DO
Umschreiben zu Funktionen.. Umsetzung für mindestens 3 Länder: Österreich, Frankreich, Italien, Deutschland, Spanien? Damit Vorhersage und Korrelation aussagekräftiger ist für mehr Daten

In [15]:
mandates_data_path = os.path.join(base_data_path, 'country_characteristics')

In [16]:
characteristics_list = []

#First the mortality rates for Austria between 2014 and 2020
mortality_at_data = infant_mortality_data[(infant_mortality_data.index.get_level_values('country') == 'austria') & (infant_mortality_data.index.get_level_values('year') >= 2014)].copy()
mortality_at_data = mortality_at_data.droplevel(1)
mortality_at_data = mortality_at_data.loc[mortality_at_data.index.repeat(8)]
mortality_at_data = mortality_at_data.reset_index()
characteristics_list.append(mortality_at_data)

#Now the education level for Austria, highest education level achieved in group 18-64, between 2014 and 2020
education_at_data = education_data[(education_data.index.get_level_values('country') == 'austria') & (education_data.index.get_level_values('year') >= 2014) & (education_data.index.get_level_values('year') < 2021)].copy()
education_at_data = education_at_data.droplevel(1)
education_at_data = education_at_data.loc[education_at_data.index.repeat(8)]
education_at_data = education_at_data.reset_index()
characteristics_list.append(education_at_data)

#And last the vaccine mandates for Austria between the times 2014 and 2020
df = pd.read_csv(os.path.join(mandates_data_path, "mandates.csv"), sep=',')
df['country'] = df['country'].str.lower()
df.set_index(['year', 'country'], inplace=True)

mandates_at_data = df[(df.index.get_level_values('country') == 'austria') & (df.index.get_level_values('year') >= 2014) & (df.index.get_level_values('year') < 2021)].copy()
mandates_at_data = mandates_at_data.droplevel(1)
mandates_at_data = mandates_at_data.reset_index()
mandates_at_data.rename(columns={'year': 'year_m'}, inplace=True) 
characteristics_list.append(mandates_at_data)


In [17]:
#Data for building our prediction and correlation model
immunization_at_data = immunization_data[(immunization_data.index.get_level_values('country') == 'austria')].reset_index()
immunization_at_data = immunization_at_data.loc[(immunization_at_data['year'] >= 2014) & (immunization_at_data['year'] < 2021)]
immunization_at_data = immunization_at_data.drop(labels='country', axis=1)
immunization_at_data = immunization_at_data.sort_values(by="year")
cols = immunization_at_data.columns[1:]

characteristics_data = pd.concat(characteristics_list, axis=1, ignore_index=False)
characteristics_data = characteristics_data.drop(labels=['year'], axis=1)
characteristics_data = characteristics_data.sort_values(by='year_m')

for el in cols:
    df_el = immunization_at_data[['year', el]].dropna()
    for index, row in df_el.iterrows():
        characteristics_data.loc[(characteristics_data['IndicatorCode'] == el.upper()) & (characteristics_data['year_m'] == int(row['year'])), 'immunization'] = row[el]

display(characteristics_data.head())


,infant_mortality,education_level,year_m,IndicatorCode,recommended,mandatory,funded,immunization
55,2.97,80.3,2014,ROTAC,1,0,1,61.0
53,2.97,80.3,2014,PCV3,1,0,1,NaN
52,2.97,80.3,2014,WHS8_110,1,0,1,96.0
54,2.97,80.3,2014,WHS4_544,1,0,1,98.0
6,3.10,78.3,2014,WHS4_543,0,0,0,NaN


## 2. Merge Data

In [18]:
def merge_data(immunization_data: pd.DataFrame, infant_mortality_data: pd.DataFrame, education_data: pd.DataFrame) -> pd.DataFrame:
    merged_data = pd.merge(immunization_data, infant_mortality_data, how='inner', left_index=True, right_index=True)
    merged_data = pd.merge(merged_data, education_data, how='inner', left_index=True, right_index=True)

    return merged_data

In [19]:
data_merged = merge_data(immunization_data, infant_mortality_data, education_data)
display(data_merged)
# printDataDimensions(data_merged)


WHS4_100  WHS4_129  WHS4_117  WHS8_110  WHS4_544  ROTAC  \
year country                                                                
2004 austria          83.0      83.0      83.0      74.0      83.0    NaN   
     malta            89.0      83.0      85.0      94.0      89.0    NaN   
     ireland          89.0      89.0       NaN      81.0      89.0    NaN   
     netherlands      98.0      97.0       NaN      96.0      98.0    NaN   
     france           98.0      97.0      35.0      88.0      99.0    NaN   
...                    ...       ...       ...       ...       ...    ...   
2020 france           96.0      95.0      91.0      92.0      96.0    NaN   
     cyprus           96.0      92.0      94.0      86.0      96.0    NaN   
     slovenia         95.0      95.0       NaN      94.0      95.0    NaN   
     spain            94.0      98.0      94.0      96.0      94.0    NaN   
     netherlands      94.0      94.0      93.0      94.0      94.0    NaN   

                  PCV3  WHS4_543  infant_mortality  education_level  
year country                                                         
2004 austria       NaN       NaN              4.21             72.5  
     malta         NaN       NaN              6.20             24.0  
     ireland       NaN      30.0              4.70             58.6  
     netherlands   NaN       NaN              4.62             64.2  
     france        NaN      85.0              3.44             59.3  
...                ...       ...               ...              ...  
2020 france       92.0       NaN              3.45             75.1  
     cyprus       81.0       NaN              2.25             75.9  
     slovenia     70.0       NaN              1.76             83.7  
     spain        94.0       NaN              2.71             58.0  
     netherlands  93.0       NaN              3.58             73.2  

[562 rows x 10 columns]

## 3 Visualization

## 4 Immunization rate prediction model

### Implementing Logistic Regression

In [20]:
#Correlation
corr_data = characteristics_data.drop(labels='year_m', axis=1).copy()
corr_data = corr_data[corr_data['IndicatorCode'] != 'PCV3']

display(corr_data.corr())


C:\Users\bamio\AppData\Local\Temp\ipykernel_15108\1191807028.py:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  display(corr_data.corr())


,infant_mortality,education_level,recommended,mandatory,funded,immunization
infant_mortality,1.000000,-0.852058,-0.797942,NaN,-0.797942,-0.077465
education_level,-0.852058,1.000000,0.734155,NaN,0.734155,0.118443
recommended,-0.797942,0.734155,1.000000,NaN,1.000000,NaN
mandatory,NaN,NaN,NaN,NaN,NaN,NaN
funded,-0.797942,0.734155,1.000000,NaN,1.000000,NaN
immunization,-0.077465,0.118443,NaN,NaN,NaN,1.000000


In [21]:
from sklearn.linear_model import LogisticRegression
X = corr_data.drop(labels=['immunization', 'IndicatorCode'], axis=1).to_numpy()
y = corr_data['immunization'].fillna(0, inplace=False).to_numpy()

training_X = X[2:]
testing_X = X[0:2]
training_y = y[2:]
testing_y = y[0:2]

model = LogisticRegression(random_state=0).fit(training_X, training_y)
print(f"True immunization: {testing_y}")
print(model.predict(testing_X))


True immunization: [61. 96.]
[85. 85.]


C:\Users\bamio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
